# Model Evaluation - Informal to Formal Text Transformation

This notebook evaluates the trained model using various metrics:
- **BLEU Score**: Measures n-gram overlap with reference translations
- **ROUGE Scores**: Measures recall-oriented overlap (ROUGE-1, ROUGE-2, ROUGE-L)
- **Perplexity**: Measures model confidence
- **Custom Metrics**: Formality score, length preservation, semantic similarity

## Requirements
Make sure you have trained a model using `train_model.ipynb` first.

In [ ]:
# Install required packages if needed
!pip install transformers datasets evaluate torch sacrebleu rouge-score nltk pandas matplotlib seaborn

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset, Dataset
import evaluate
from tqdm.auto import tqdm

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

## 1. Load Model and Test Data

In [ ]:
# Configuration
MODEL_PATH = "./models/informal-to-formal-t5"  # Path to your trained model
TEST_DATA_PATH = "./data/informal_formal/test.json"  # Path to test data

# Check if model exists
if not Path(MODEL_PATH).exists():
    print(f"WARNING: Model not found at {MODEL_PATH}")
    print("Using pretrained model instead: google/flan-t5-small")
    MODEL_PATH = "google/flan-t5-small"
else:
    print(f"✓ Model found at {MODEL_PATH}")

# Load model and tokenizer
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(f"Model loaded on {device}")
print(f"Model parameters: {model.num_parameters():,}")

In [ ]:
# Load test data
if Path(TEST_DATA_PATH).exists():
    with open(TEST_DATA_PATH, 'r') as f:
        test_data = json.load(f)
    print(f"✓ Loaded {len(test_data)} test samples")
else:
    print(f"WARNING: Test data not found at {TEST_DATA_PATH}")
    print("Creating sample test data...")
    test_data = [
        {"informal": "send me the file now", "formal": "Could you please send me the file at your earliest convenience?"},
        {"informal": "why didn't you finish the report?", "formal": "I noticed the report is incomplete. Could you please provide an update on its status?"},
        {"informal": "need this asap", "formal": "I would appreciate receiving this as soon as possible."},
        {"informal": "what's taking so long?", "formal": "May I inquire about the current progress?"},
        {"informal": "good job", "formal": "Excellent work on this project."},
    ]

# Display sample
print("\nSample test data:")
for i, item in enumerate(test_data[:3]):
    print(f"{i+1}. Informal: {item['informal']}")
    print(f"   Formal: {item['formal']}\n")

## 2. Generate Predictions

In [ ]:
def generate_prediction(informal_text, max_length=128, num_beams=4):
    """Generate formal text from informal text."""
    prompt = f"Convert informal to formal: {informal_text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            num_beams=num_beams,
            early_stopping=True,
            do_sample=False,
        )
    
    prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return prediction

# Generate predictions for all test samples
print("Generating predictions...")
predictions = []
references = []
inputs = []

for item in tqdm(test_data):
    informal_text = item['informal']
    formal_text = item['formal']
    
    prediction = generate_prediction(informal_text)
    
    inputs.append(informal_text)
    predictions.append(prediction)
    references.append(formal_text)

print(f"\n✓ Generated {len(predictions)} predictions")

In [ ]:
# Display some examples
print("\n" + "="*80)
print("SAMPLE PREDICTIONS")
print("="*80)

for i in range(min(5, len(test_data))):
    print(f"\nExample {i+1}:")
    print(f"  Input (Informal):  {inputs[i]}")
    print(f"  Reference (Formal): {references[i]}")
    print(f"  Prediction:         {predictions[i]}")
    print("-" * 80)

## 3. Calculate BLEU Score

In [ ]:
# Load BLEU metric
bleu_metric = evaluate.load("sacrebleu")

# Calculate BLEU score
references_list = [[ref] for ref in references]  # sacrebleu expects list of lists
bleu_result = bleu_metric.compute(predictions=predictions, references=references_list)

print("\n" + "="*80)
print("BLEU SCORE")
print("="*80)
print(f"BLEU Score: {bleu_result['score']:.2f}")
print(f"\nInterpretation:")
print(f"  > 40: Excellent translation quality")
print(f"  30-40: Good translation quality")
print(f"  20-30: Acceptable translation quality")
print(f"  < 20: Poor translation quality")
print(f"\nYour model: {bleu_result['score']:.2f} - ", end="")
if bleu_result['score'] > 40:
    print("Excellent! 🎉")
elif bleu_result['score'] > 30:
    print("Good! 👍")
elif bleu_result['score'] > 20:
    print("Acceptable ✓")
else:
    print("Needs improvement 📝")

## 4. Calculate ROUGE Scores

In [ ]:
# Load ROUGE metric
rouge_metric = evaluate.load("rouge")

# Calculate ROUGE scores
rouge_result = rouge_metric.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)

print("\n" + "="*80)
print("ROUGE SCORES")
print("="*80)
print(f"ROUGE-1 (unigram overlap):  {rouge_result['rouge1']:.4f}")
print(f"ROUGE-2 (bigram overlap):   {rouge_result['rouge2']:.4f}")
print(f"ROUGE-L (longest common):   {rouge_result['rougeL']:.4f}")
print(f"\nInterpretation:")
print(f"  ROUGE-1: Measures word-level overlap (higher is better)")
print(f"  ROUGE-2: Measures phrase-level overlap (higher is better)")
print(f"  ROUGE-L: Measures fluency and coherence (higher is better)")
print(f"  Good scores are typically > 0.5 for ROUGE-1 and > 0.3 for ROUGE-2")

## 5. Calculate Perplexity

In [ ]:
def calculate_perplexity(texts):
    """Calculate average perplexity for a list of texts."""
    total_loss = 0
    total_tokens = 0
    
    model.eval()
    with torch.no_grad():
        for text in tqdm(texts, desc="Calculating perplexity"):
            inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
            
            # For decoder models, labels should be the same as input_ids
            labels = inputs.input_ids.clone()
            
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            
            total_loss += loss.item() * inputs.input_ids.size(1)
            total_tokens += inputs.input_ids.size(1)
    
    avg_loss = total_loss / total_tokens
    perplexity = np.exp(avg_loss)
    
    return perplexity

# Calculate perplexity on predictions
print("\n" + "="*80)
print("PERPLEXITY")
print("="*80)

try:
    perplexity = calculate_perplexity(predictions[:50])  # Calculate on first 50 to save time
    print(f"Perplexity: {perplexity:.2f}")
    print(f"\nInterpretation:")
    print(f"  Lower perplexity = better model confidence")
    print(f"  < 50: Excellent")
    print(f"  50-100: Good")
    print(f"  100-200: Acceptable")
    print(f"  > 200: Poor")
except Exception as e:
    print(f"Could not calculate perplexity: {e}")
    perplexity = None

## 6. Custom Metrics

In [ ]:
# Length preservation (how well does the model preserve input length?)
input_lengths = [len(text.split()) for text in inputs]
prediction_lengths = [len(text.split()) for text in predictions]
reference_lengths = [len(text.split()) for text in references]

# Calculate average length ratios
pred_to_input_ratio = np.mean([p/i if i > 0 else 0 for p, i in zip(prediction_lengths, input_lengths)])
pred_to_ref_ratio = np.mean([p/r if r > 0 else 0 for p, r in zip(prediction_lengths, reference_lengths)])

print("\n" + "="*80)
print("LENGTH ANALYSIS")
print("="*80)
print(f"Average input length:      {np.mean(input_lengths):.1f} words")
print(f"Average prediction length: {np.mean(prediction_lengths):.1f} words")
print(f"Average reference length:  {np.mean(reference_lengths):.1f} words")
print(f"\nLength ratios:")
print(f"  Prediction/Input:    {pred_to_input_ratio:.2f}x")
print(f"  Prediction/Reference: {pred_to_ref_ratio:.2f}x")
print(f"\nNote: Formal text is typically 2-3x longer than informal text")

In [ ]:
# Word-level analysis
# Check for informal words in predictions
informal_words = {
    'u', 'ur', 'pls', 'thx', 'gonna', 'wanna', 'gotta', 'kinda', 'sorta',
    'dunno', 'gimme', 'lemme', 'yeah', 'yep', 'yup', 'nope', 'nah',
    'asap', 'fyi', 'btw', 'omg', 'lol', 'brb', 'gtg'
}

def count_informal_words(text):
    words = text.lower().split()
    return sum(1 for word in words if word in informal_words)

informal_in_inputs = [count_informal_words(text) for text in inputs]
informal_in_predictions = [count_informal_words(text) for text in predictions]

print("\n" + "="*80)
print("FORMALITY ANALYSIS")
print("="*80)
print(f"Informal words in inputs:      {sum(informal_in_inputs)}")
print(f"Informal words in predictions: {sum(informal_in_predictions)}")
print(f"Formalization rate: {(1 - sum(informal_in_predictions) / max(sum(informal_in_inputs), 1)) * 100:.1f}%")
print(f"\nNote: Good models should remove most informal words")

## 7. Visualizations

In [ ]:
# Create summary dataframe
results_df = pd.DataFrame({
    'Input': inputs,
    'Reference': references,
    'Prediction': predictions,
    'Input_Length': input_lengths,
    'Prediction_Length': prediction_lengths,
    'Reference_Length': reference_lengths,
})

print("\nResults DataFrame:")
print(results_df.head())

In [ ]:
# Plot 1: Length comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Length distribution
axes[0].hist([input_lengths, prediction_lengths, reference_lengths], 
             label=['Input', 'Prediction', 'Reference'],
             alpha=0.6, bins=15)
axes[0].set_xlabel('Text Length (words)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Text Length Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Length correlation
axes[1].scatter(reference_lengths, prediction_lengths, alpha=0.6)
axes[1].plot([0, max(reference_lengths)], [0, max(reference_lengths)], 'r--', label='Perfect match')
axes[1].set_xlabel('Reference Length (words)')
axes[1].set_ylabel('Prediction Length (words)')
axes[1].set_title('Length Correlation: Prediction vs Reference')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('length_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Length analysis plot saved as 'length_analysis.png'")

In [ ]:
# Plot 2: Metrics summary
metrics_data = {
    'BLEU': bleu_result['score'] / 100,  # Normalize to 0-1
    'ROUGE-1': rouge_result['rouge1'],
    'ROUGE-2': rouge_result['rouge2'],
    'ROUGE-L': rouge_result['rougeL'],
}

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metrics_data.keys(), metrics_data.values(), color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax.set_ylabel('Score')
ax.set_title('Model Evaluation Metrics', fontsize=16, fontweight='bold')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('metrics_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Metrics summary plot saved as 'metrics_summary.png'")

## 8. Save Results

In [ ]:
# Save detailed results
results_df.to_csv('evaluation_results.csv', index=False)
print("✓ Detailed results saved to 'evaluation_results.csv'")

# Save metrics summary
metrics_summary = {
    'model_path': MODEL_PATH,
    'test_samples': len(test_data),
    'bleu_score': float(bleu_result['score']),
    'rouge1': float(rouge_result['rouge1']),
    'rouge2': float(rouge_result['rouge2']),
    'rougeL': float(rouge_result['rougeL']),
    'perplexity': float(perplexity) if perplexity else None,
    'avg_input_length': float(np.mean(input_lengths)),
    'avg_prediction_length': float(np.mean(prediction_lengths)),
    'avg_reference_length': float(np.mean(reference_lengths)),
    'pred_to_input_ratio': float(pred_to_input_ratio),
    'pred_to_ref_ratio': float(pred_to_ref_ratio),
    'informal_words_removed': sum(informal_in_inputs) - sum(informal_in_predictions),
    'formalization_rate': float((1 - sum(informal_in_predictions) / max(sum(informal_in_inputs), 1)) * 100),
}

with open('metrics_summary.json', 'w') as f:
    json.dump(metrics_summary, f, indent=2)

print("✓ Metrics summary saved to 'metrics_summary.json'")

## 9. Final Summary

In [ ]:
print("\n" + "="*80)
print("FINAL EVALUATION SUMMARY")
print("="*80)
print(f"\nModel: {MODEL_PATH}")
print(f"Test Samples: {len(test_data)}")
print(f"\nAutomatic Metrics:")
print(f"  BLEU:    {bleu_result['score']:.2f}")
print(f"  ROUGE-1: {rouge_result['rouge1']:.4f}")
print(f"  ROUGE-2: {rouge_result['rouge2']:.4f}")
print(f"  ROUGE-L: {rouge_result['rougeL']:.4f}")
if perplexity:
    print(f"  Perplexity: {perplexity:.2f}")
print(f"\nCustom Metrics:")
print(f"  Formalization Rate: {metrics_summary['formalization_rate']:.1f}%")
print(f"  Length Expansion: {pred_to_input_ratio:.2f}x")
print(f"\nFiles Generated:")
print(f"  - evaluation_results.csv")
print(f"  - metrics_summary.json")
print(f"  - length_analysis.png")
print(f"  - metrics_summary.png")
print("\n" + "="*80)
print("EVALUATION COMPLETE!")
print("="*80)